In [1]:
import numpy as np
import pandas as pd
import torch as t
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import time, datetime

template = "plotly_dark"
from copy import deepcopy


def time_now():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [2]:
# Create some sort of resemblance to
SLIT_SIZE = 50
# number of particles are defined by the slit size.
N_PARTICLES = SLIT_SIZE * 100

# Wall parameters
WALL_Y = 0
WALL_LENGTH = 0

PARTICLES_Y = 0
Y_LIMIT = 2000
Y_VELOCITY = 300_000
G = 66260
# G = 5_000_000
MASS = 1
X_LIMIT = 50
# how many decimals to round x for plotting
X_ROUND = 0

# Related to wave part
wavegrav = True
wavelength = 10
type_sin_cos = "sin"
POWER = 1
INNER_POWER = 1


# Calculations

In [3]:
p_x = t.linspace(-SLIT_SIZE // 2, SLIT_SIZE // 2, N_PARTICLES + 1)
p_y = t.zeros_like(p_x) + PARTICLES_Y
p_xy = t.stack((p_x, p_y), 1).cuda()
trail_xy = t.clone(p_xy.unsqueeze(0)).cuda()
v_xy = t.zeros_like(p_xy).cuda() + t.tensor((0, Y_VELOCITY)).cuda()
dt = t.ones((N_PARTICLES + 1, 1)).cuda()
p_mass = t.ones_like(dt).cuda()

# 2 walls, 1 slit
o_x_negatives = t.linspace(
    -SLIT_SIZE // 2 - WALL_LENGTH, -SLIT_SIZE // 2, WALL_LENGTH + 1
)
o_x_positives = t.linspace(
    SLIT_SIZE // 2, SLIT_SIZE // 2 + WALL_LENGTH, WALL_LENGTH + 1
)
o_x = t.cat((o_x_negatives, o_x_positives), 0)
o_y = t.zeros_like(o_x) + WALL_Y
o_xy = t.stack((o_x, o_y), 1).cuda()

for y in range(Y_LIMIT):
    dt = t.ones((N_PARTICLES + 1, 1)).cuda()
    a_xy = t.zeros_like(p_xy).cuda()
    for i in o_xy:
        # distances
        d_xy = p_xy - i
        r = t.sqrt(t.sum(d_xy**2, axis=1))
        r_ = r * wavelength
        a = -1 * G * MASS / r**2
        if wavegrav:
            if type_sin_cos == "sin":
                a *= t.sin(r_**INNER_POWER) ** (POWER)
            else:
                a *= t.cos(r_**INNER_POWER) ** (POWER)
        a_xy += a.unsqueeze(1) * d_xy / t.stack([r, r], axis=1)

    v_xy += a_xy * dt
    dt = 1 / v_xy
    dt[dt == float("Inf")] = 0
    dt = dt[:, 1].unsqueeze(1)
    t.abs_(dt)
    p_xy += v_xy * dt
    trail_xy = t.cat((trail_xy, p_xy.unsqueeze(0)), 0)
trails = trail_xy.reshape(-1, 2).cpu().numpy().round(X_ROUND)


In [4]:
grid = pd.DataFrame(trails, columns=["x", "y"])
grid_pt = grid[(grid.x > -X_LIMIT) & (grid.x < X_LIMIT)].pivot_table(
    index="y", columns="x", aggfunc=len, fill_value=0
)
grid.dropna(inplace=True)


In [5]:
wall_left = np.array([[-X_LIMIT, -SLIT_SIZE // 2], [WALL_Y, WALL_Y], [0, 0]])
wall_right = np.array([[SLIT_SIZE // 2, X_LIMIT], [WALL_Y, WALL_Y], [0, 0]])


# Plotting

In [6]:
params = {
    "SLIT_SIZE": SLIT_SIZE,
    "wavelength": wavelength,
    "type_sin_cos": type_sin_cos,
    "INNER_POWER": INNER_POWER,
    "POWER": POWER,
    "N_PARTICLES": N_PARTICLES,
    "G": G,
    "WALL_Y": WALL_Y,
    "WALL_LENGHT": WALL_LENGTH,
    "PARTICLES_Y": PARTICLES_Y,
    "Y_LIMIT": Y_LIMIT,
    "MASS": MASS,
    "X_LIMIT": X_LIMIT,
    "X_ROUND": X_ROUND,
    "wavegrav": wavegrav,
    "Y_VELOCITY": Y_VELOCITY,
}


In [44]:
wall_left


array([[-50, -25],
       [  0,   0],
       [  0,   0]])

In [49]:
SLIT_SIZE // 2


25

In [54]:
# Implement model parameters in title
fig = go.Figure()

fig.add_trace(
    go.Surface(
        z=grid_pt.values / grid_pt.values.max(),
        x=grid_pt.columns.tolist(),
        y=grid_pt.index.tolist(),
        showscale=False,
        hoverinfo=None,
        reversescale=True,
        name="test",
        # colorscale=["red", "black"],
        colorscale="Turbo",
        lighting=dict(diffuse=0.0, specular=0.0, fresnel=0.0, roughness=0.0),
        # cmin=0,
        # cmax=500,
        contours_z=dict(
            project_x=False,
            project_y=False,
            project_z=False,
            highlight=True,
            highlightcolor="red",
            highlightwidth=1,
        ),
        contours_y=dict(
            highlightcolor="red",
            project_y=True,
            project_z=False,
            project_x=False,
            highlight=True,
            width=1,
            highlightwidth=1,
            show=False,
        ),
        contours_x=dict(
            highlightcolor="red",
            project_x=True,
            project_y=False,
            project_z=False,
            highlight=False,
            width=16,
            show=False,
            highlightwidth=1,
        ),
    )
)

fig.add_trace(
    go.Mesh3d(
        x=[-X_LIMIT, -SLIT_SIZE // 2, -SLIT_SIZE // 2, -X_LIMIT],
        y=[WALL_Y, WALL_Y, WALL_Y, WALL_Y],
        z=[0, 0, 10, 10],
        color="white",
        opacity=0.5,
        hoverinfo=None,
        name="left_wall",
    )
)

# fig.add_trace(
#     go.Scatter3d(
#         x=wall_right[0],
#         y=wall_right[1],
#         z=wall_right[2],
#         marker_color="white",
#         mode="lines",
#         hoverinfo=None,
#         name="left_wall",
#     )
# )
# fig.add_trace(
#     go.Scatter3d(
#         x=wall_left[0],
#         y=wall_left[1],
#         z=wall_left[2],
#         marker_color="white",
#         mode="lines",
#         hoverinfo=None,
#         name="left_wall",
#     )
# )
fig.update_layout(template=template)
fig.update_layout(
    title=str(params),
    font=dict(size=8),
    scene=dict(
        xaxis_showspikes=False,
        yaxis_showspikes=False,
        zaxis_showspikes=False,
        xaxis_visible=False,
        yaxis_visible=False,
        zaxis_visible=False,
        yaxis_range=[0, grid_pt.index.to_numpy().max() * 1.5],
    ),
    margin=dict(t=50, b=0, l=0, r=0),
)

fig.write_html(f"plotly_graphs/simulator/notebook_surface.html")
fig.write_html(f"plotly_graphs/simulator/notebook_surface_{time_now()}.html")
